In [6]:
import sys
import os
# Add the project root directory to sys.path
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../')))

from pos.database import Database
import pandas as pd
from pprint import pprint
from collections import defaultdict
from datetime import datetime

db = Database()


--> Database connection established successfully!


In [7]:
data = db.fetch_all_sales()

# Get daily sales
daily_sales = defaultdict(float)
for sale in data:
    date = sale["date"].strftime('%Y-%m-%d')
    daily_sales[date] += float(sale["total_amount"])

daily_sale_data = [{"date": datetime.strptime(date, '%Y-%m-%d'), "total_amount": amount} for date, amount in daily_sales.items()]


In [2]:
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle
from reportlab.lib.pagesizes import landscape
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib import colors
import os


class InvoiceGenerator:
    def __init__(self, invoice_id, sale_data, sale_items):
        self.invoice_id = invoice_id
        self.sale_data = sale_data
        self.sale_items = sale_items

    def generate_invoice(self, filename="invoice.pdf"):
        # Page size for a small thermal paper
        page_width, page_height = (58, 100)  # Thermal printer paper size in points
        doc = SimpleDocTemplate(
            filename,
            pagesize=(page_width, page_height),
            rightMargin=2,
            leftMargin=2,
            topMargin=2,
            bottomMargin=2,
        )
        elements = []

        # Styles for the document
        styles = getSampleStyleSheet()
        title_style = styles['Title']
        normal_style = styles['Normal']

        # Adjust font sizes for small paper
        title_style.fontSize = 6
        normal_style.fontSize = 4

        # Add header
        elements.append(Paragraph("TOKATA - Invoice", title_style))
        elements.append(Spacer(1, 4))
        elements.append(Paragraph(f"Invoice Number: {self.invoice_id}", normal_style))
        elements.append(Paragraph(f"Date: {self.sale_data['date']}", normal_style))
        elements.append(Spacer(1, 4))

        # Add table for items
        table_data = [["Item", "Qty", "Price", "Total"]]
        total = 0
        for item in self.sale_items:
            item_total = item['quantity'] * item['unit_price']
            table_data.append([
                Paragraph(item['name'], normal_style),  # Use Paragraph for wrapping long text
                item['quantity'],
                f"${item['unit_price']:.2f}",
                f"${item_total:.2f}",
            ])
            total += item_total

        # Add totals row
        table_data.append(["", "", "Total", f"${total:.2f}"])

        # Create table
        table = Table(
            table_data,
            colWidths=[20, 8, 15, 15],  # Adjust column widths for better fit
            hAlign='LEFT',
        )
        table.setStyle(TableStyle([
            ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
            ('TEXTCOLOR', (0, 0), (-1, 0), colors.white),
            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
            ('FONTNAME', (0, 0), (-1, -1), 'Helvetica'),
            ('FONTSIZE', (0, 0), (-1, -1), 3.5),  # Smaller font size for table
            ('BOTTOMPADDING', (0, 0), (-1, -1), 1),
            ('GRID', (0, 0), (-1, -1), 0.1, colors.black),
        ]))
        elements.append(table)
        elements.append(Spacer(1, 6))

        # Add footer
        elements.append(Paragraph("Thank you for your purchase!", normal_style))
        elements.append(Spacer(1, 2))
        elements.append(Paragraph("© TOKATA. All rights reserved.", normal_style))

        # Build PDF
        doc.build(elements)
        print(f"Invoice generated: {os.path.abspath(filename)}")


# Example usage
if __name__ == "__main__":
    # Dummy data
    invoice_id = "INV-001"
    sale_data = {
        "date": "2024-11-24",
    }
    sale_items = [
        {"name": "Item A", "quantity": 2, "unit_price": 5.50},
        {"name": "Item B", "quantity": 1, "unit_price": 12.00},
        {"name": "Item C", "quantity": 3, "unit_price": 3.75},
    ]

    # Generate invoice
    generator = InvoiceGenerator(invoice_id, sale_data, sale_items)
    generator.generate_invoice("invoice.pdf")


Invoice generated: /Users/home/Desktop/pos_system/test/invoice.pdf
